# Devoir d'astroparticules 

Noter que les premiers codes proviennent de la séance 2.
## Rappels

In [118]:
Z = 26
A = 55.85
Ec= 5700/(Z+1.47)**0.838 # in GeV
alpha = 0.002 #GeV/(g/cm2)
beta = alpha/Ec #1/(g/cm2)

In [119]:
rho = 7.87 #g/cm^3
Rdet = 4. * 6 * rho #Detection range in g/cm2

def muon_backprop(emu, X, a, b):

    """
    Calculate the original muon energy given an energy and amount
    of material crossed
    Inputs:
        - emu: current energy
        - X: material crossed
        - a: ion energy loss coef.
        - b: radiaion energy loss coef.
    """
    ec = a/b
    return (emu + ec)*np.exp(b*X) - ec

#The Energy thershold is the minimum energy needed to cross 4 layers of Iron, ie when E_mu(R) = 0, or X = R:
Eth = muon_backprop(0, Rdet, alpha, beta)

In [120]:
X0 = 716.4 * A /(Z*(Z+1) * np.log(287/np.sqrt(Z))) # g/cm2
Ec = 21 *1e-3 # GeV
EMth = Ec * 2**(Rdet/X0)

In [121]:
def muons(E, cangle):
    fpion = 1./(1.+ 1.1*E*cangle/115.)
    fkaon = 0.054/(1.+ 1.1*E*cangle/850.)
    return 0.14 *E**-2.7 *(fpion + fkaon)

In [122]:
e = np.logspace(0, 6, 51)
cangle = np.cos(np.radians(50))
X0 = 1.2E5 * 2.65 # Distance in g/cm^2

a = 1.677e-3 # GeV cm^2/g
b = 3.8e-6 # cm^2/g

In [123]:
dcangle = 0.1 #steps of 0.1 in cosine

cz = np.arange(np.cos(np.radians(70.)), 1, dcangle)
e = np.logspace(0, 6, 200) #in GeV
ntop = 0 #flux at the top surface
nsid = 0 #flux at the sides surfaces

atop0 = 1600 * 4000 #Top area in cm2
aside0 = 2 * 140 * 8.5 * (1600 + 4000) #Side area in cm2

#Let's do the integral both in energy and angle.

for i, energy in enumerate(e[:-1]):
    de = e[i+1] -e[i]
    for cangle in cz:
        X = X0/cangle
        flux = muons(muon_backprop(energy, X, a, b), cangle) * de * dcangle
        
        #Number of muon at the top surface is:
        # flux * 2pi * projected_area
        ntop += flux * 2 * np.pi * (cangle * atop0)
        
        #Number of muon at the top surface is:
        # flux * 2pi * projected_area
        nsid += flux * 2 * np.pi * np.sqrt(1 - cangle**2) * aside0

## Devoir

Nous allons trouver le taux d'évènement de neutrino atmosphérique ($\nu_\mu \rightarrow \mu^-$ et $\bar{\nu}_\mu \rightarrow \mu^+$) qui interagissent via une interaction CC. Pour ce faire, nous allons utiliser le flux de Honda. Il faut télécharger le fichier correspoondant au site INO avec montagne et maximum solaire. L'analyse suivante est faite sans oscillation de neutrino.

Une paramétrisation utile des sections efficaces de neutrino CC comme une fonction de l'énergie du lepton sortant est donnée par Gaisser et Stanev (Phys. Rev. D30 (1984) 985):
$$\frac{d\sigma_\nu}{dE_\mu} = \left[ 0.72+0.06\left(\frac{E_\mu}{E_\nu}\right)^2\right] \times 10^{-38}cm^2GeV^{-1}$$
$$\frac{d\sigma_\bar{\nu}}{dE_\mu} = \left[ 0.09+0.69\left(\frac{E_\mu}{E_\nu}\right)^2\right] \times 10^{-38}cm^2GeV^{-1}$$
Pour la section efficace totale de neutrino utilisée pour calculer "Earth Shadow" on peut utiliser $\sigma_{\nu p} \simeq 0.69 \times 10^{-38}E_\nu cm^2 $

Pour télécharger les fichiers de Honda, on utilise Python, qui va directement télécharger la ressource depuis le web. 

In [124]:
import gzip
import urllib.request

url = "http://www.icrr.u-tokyo.ac.jp/~mhonda/nflx2014/ino-ally-20-01-mtn-solmax.d.gz"
file = urllib.request.urlopen(url)
with open("honda.gz", "wb") as local_file:
    local_file.write(file.read())
    
inF = gzip.GzipFile("honda.gz", 'rb')
s = inF.read()
inF.close()

outF = open("honda.d", 'wb')
outF.write(s)
outF.close()

Vérifions que le fichier est bien là:

In [125]:
from IPython.display import FileLink, FileLinks
FileLink('honda.d')

/Users/maxvandenbemden/Desktop/ULB/MASTER/ASTROPARTICULES/honda.d

En regardant le fichier on voit que le format est fort complexe, les données sont séparées en 20 blocs d'angle allant de $\cos\theta = -1$ jusque $\cos\theta = 1$. Chacun de ces blocs d'angle contient 101 entrées qui correspondent à des énergies allant de 0.1 GeV à $10^4$ GeV.

Comme le format du fichier est fort complexe, nous devons créer notre propre code pour lire les données. 

In [126]:
class HondaFlux(object):
    """Honda average over azimuth"""
    def __init__(self, f, name):
        state = 0
        self.name = name
        self.elog = np.linspace(-1, 4, 101)
        self.cz = np.linspace(1, -1, 20)
        self.numu = np.zeros((20, 101))
        self.numubar = np.zeros((20, 101))
        self.nue = np.zeros((20, 101))
        self.nuebar = np.zeros((20, 101))
        block = 0
        for line in f:
            state += 1
            if state == 1:
                cmin = float(line[23:28])
                cmax = float(line[32:37])
                amin = float(line[48:51])
                amax = float(line[54:58])
            elif state > 2:
                energy, numu, numubar, nue, nuebar = [ float(x) for x in line.split() ]
                i = block
                k = state - 3
                self.numu[i,k] = numu
                self.numubar[i,k] = numubar
                self.nue[i,k] = nue
                self.nuebar[i,k] = nuebar
            if state == 103:
                state = 0
                block += 1

Une estimation du taux de détection des évènements neutrinos est équivalent à calculer le taux de flux de muons induits par des neutrinos:
$$R(E_{vis},\theta) = \int_{E_{vis}} P_{\nu \rightarrow l}(E_\nu, E_{vis})P_{shadow}(\theta, E_\nu)\frac{dN_\nu}{dE_\nu}dE_\nu$$
Avec $P_{\nu \rightarrow l}(E_\nu, E_{vis})$, la probabilité qu'un neutrino interagisse avec un noyau pour produire un muon ou un EM ou une cascade hadronique avec une énergie minimale $E_{vis}$ visible dans le détecteur. $P_{shadow}(\theta, E_\nu)$, la probabilité qu'un neutrino avec un angle de zénith $\theta$ et une énergie $E_\nu$ d'être absorbé par la Terre. $\frac{dN_\nu}{dE_\nu}$ le flux deneutrino à la surface.

In [127]:
import astropy.constants as cte
rho_r = 2.65 #g/cm^3 #la densité de la roche
R_E = cte.R_earth.to("cm").value #le rayon de la Terre
d = 1200 * 1e2 # la profondeur du détecteur en cm

Avec l'exercice 3 on peut construire $X(\theta)$

In [128]:
def X(theta):
    if theta == 0:
        return rho_r * d 
    elif theta == np.pi:
        return rho_r * (2 * R_E - d) 
    else: # par l'exercice 3 on a:
        gamma = np.pi - theta 
        beta = np.arcsin((R_E - d)/R_E * np.sin(gamma))
        alpha = np.pi - gamma - beta
        return rho_r * np.sin(alpha)/np.sin(gamma) * R_E #g/cm^2

En utilisant la section efficace totale de l'énoncé on a:

In [129]:
def Pshadow(theta, E): 
        return np.exp(-X(theta)*cte.N_A.value * 0.69 * 1e-38 * E)

On peut calculer la probabilité d'interaction, elle est donnée par la formule suivante.
$$P_{\nu\rightarrow \mu} = N_A \int^{E_\nu}_{E_{vis}} dE_\mu \frac{d\sigma}{dE_\mu} r_\mu(E_\mu,E_{vis})$$ Pour le neutrino muonique la portée est donnée par le muon qui arrive avec une énergie nulle dans le détecteur. On a l'équation
$$E_0 = (E(X) + \epsilon_\mu)\exp^{\beta X} - \epsilon_\mu$$ De là on peut facilement dériver la distance traversée par un muon qui comme énergie initiale $E_0$ et qui arrive à l'énergie $E(r)$.
$$r_\mu= \frac{1}{\beta}\log\frac{\epsilon + E_0}{\epsilon + E(r)}$$ Cette expression correspond bien à la portée R quand l'énergie résiduelle est nulle. 

Enfin on va utiliser la paramétrisation de la section efficace du neutrino de l'énoncé $$\frac{d\sigma_\nu}{dE_\mu} = \left[ 0.72+0.06\left(\frac{E_\mu}{E_\nu}\right)^2\right] \times 10^{-38}cm^2GeV^{-1}$$

In [130]:
HondaF = HondaFlux(open("honda.d"), "INO-SolarMax")
def rmuon(E0, Er, a, b):
    return 1/b * np.log((a/b + E0)/(a/b + Er)) # g/cm2

In [131]:
def Probint(theta, enu):
    eth = muon_backprop(0, Rdet/np.abs(np.cos(theta)), alpha, beta) #énergie de seuil pour être visible dans du Fer
    energy = np.logspace(np.log10(eth), np.log10(enu), 300)

    Prob = 0
    for i, emu in enumerate(energy[:-1]):
        deltaE = energy[i + 1] - energy[i]
        sigma = (0.72 + 0.06*(emu/enu)**2) * 1e-38 #in cm^2 GeV-1
        Prob += cte.N_A.value * sigma * rmuon(emu, eth, a, b) * deltaE
    return Prob

Le taux sera donné donc donné par une intégrale sur le produit $P_{shadow} \cdot P_{int}$ par convolution avec le flux de neutrino. 

In [138]:
from astropy import units
HondaF = HondaFlux(open("honda.d"), "INO-SolarMax")

rate = 0
for i in range(100):
    
    for j in range(19):
        
        e = 10**HondaF.elog[i]
        deltaE = (10**HondaF.elog[i+1] - 10**HondaF.elog[i]) #DeltaR
        theta = np.arccos(HondaF.cz[j])
        Psh = Pshadow(theta, e)
        Pr = Probint(theta, e)
        dcos = np.abs(HondaF.cz[j] - HondaF.cz[j+1])
        flux = HondaF.numu[j,i]
        
        
        rate += Psh * Pr * flux * deltaE * dcos

Latex(r"Le taux de $\nu_\mu$ par an est de %.2f" %(rate * 2 * np.pi * units.year.to(units.s)))        

<IPython.core.display.Latex object>

In [139]:
HondaF = HondaFlux(open("honda.d"), "INO-SolarMax")

ratebar = 0
for i in range(100):
    
    for j in range(19):
        
        e = 10**HondaF.elog[i]
        deltaE = (10**HondaF.elog[i+1] - 10**HondaF.elog[i]) #DeltaR
        theta = np.arccos(HondaF.cz[j])
        Psh = Pshadow(theta, e)
        Pr = Probint(theta, e)
        dcos = np.abs(HondaF.cz[j] - HondaF.cz[j+1])
        fluxbar = HondaF.numubar[j,i]
        
        
        ratebar += Psh * Pr * fluxbar * deltaE * dcos

Latex(r"Le taux de $\bar{\nu}_\mu$ par an est de %.2f" %(ratebar * 2 * np.pi * units.year.to(units.s)))        

<IPython.core.display.Latex object>